In [ ]:
import adaptive_latents as al

import matplotlib.pyplot as plt
import numpy as np

from prediction_regression_run import pred_reg_run_with_defaults

In [ ]:
run_without_logging = pred_reg_run_with_defaults('odoherty21', log_level=1)

In [ ]:
run_with_logging = pred_reg_run_with_defaults('odoherty21', log_level=2)


In [ ]:
%matplotlib qt


stream_label_mapping = {
    0: 'neural sample',
    1: 'behavior sample',
    2: 'target data sample',
    3: 'prediction',
    # -1: 'thrown out sample',
}


def time_scatter(ax, transformer, run, semilogy=True):
    a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
    stream = np.array(transformer.log['stream'])
    for s in stream_label_mapping:
        sub_a = a[stream == s]
        sub_a.t = sub_a.t[stream==s]
        ax.plot(sub_a.t, sub_a, '.', label=stream_label_mapping[s], alpha=1 if s == -1 else 1)

    q = np.quantile(a,.999)
    # ax.axhline(q, color='k')
    # ax.text(0, q*1.2, f'{q:.1f} ms')
    sampling_dt = run.sources[0][0].dt * 1000
    ax.axhline(sampling_dt, color='k', lw=1.5, ls='-')
    ax.set_xlabel('experiment time (s)')
    ax.set_ylabel('end-to-end iteration time (ms)')
    ax.set_title(f'step times for a "{type(transformer).__name__}" step on the {run.dataset} dataset')
    ax.legend()
    if semilogy:
        ax.semilogy()
        ax.text(0, sampling_dt*1.2, f'{sampling_dt:.1f} ms (sampling rate)')
    else:
        ax.set_ylim([0, sampling_dt*2])
        ax.text(0, sampling_dt + .2, f'{sampling_dt:.1f} ms (sampling rate)')


fig, axs = plt.subplots(ncols=2)
time_scatter(axs[0], run_without_logging.pipeline, run_without_logging, semilogy=False)
time_scatter(axs[1], run_with_logging.pipeline, run_with_logging, semilogy=False)



In [ ]:
%matplotlib inline

def row_hist_per_stream(transformer, run):
    fig, axs = plt.subplots(nrows=len(stream_label_mapping), squeeze=False, sharex=True)
    
    
    a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
    stream = np.array(transformer.log['stream'])
    sampling_dt = run.sources[0][0].dt * 1000


    for idx, s in enumerate(stream_label_mapping):
        sub_a = a[stream == s]
        sub_a.t = sub_a.t[stream==s]
        if sub_a.size:
            l,r = np.quantile(sub_a,.005), np.quantile(sub_a,.995)
            axs[idx,0].hist(sub_a, bins=np.linspace(l,r,101), color=f'C{idx}')
            axs[idx,0].semilogy()

        axs[idx,0].axvline(sampling_dt, color='k', lw=1.5, ls='-')
        axs[idx,0].set_xlim([0, sampling_dt*2])

    # ax.text(0, q*1.2, f'{q:.1f} ms')
    # ax.text(0, sampling_dt*1.2, f'{sampling_dt:.1f} ms (sampling rate)')
    axs[0,0].set_title('processing step length by stream')
    axs[-1,0].set_xlabel('step time (ms)')
    
run = run_without_logging
row_hist_per_stream(run.pipeline, run)


In [ ]:

def row_hist_per_step(pipeline, run, s):
    fig, axs = plt.subplots(nrows=len(pipeline.steps), squeeze=False, sharex=True, figsize=(4,10))
    sampling_dt = run.sources[0][0].dt * 1000

    for idx, transformer in enumerate(pipeline.steps):
            a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
            stream = np.array(transformer.log['stream'])
            sub_a = a[stream == s]
            sub_a.t = sub_a.t[stream==s]
            if sub_a.size:
                l,r = np.quantile(sub_a,.0005), np.quantile(sub_a,.9995)
                axs[idx,0].hist(sub_a, bins=np.linspace(l,r,101), color=f'C{idx}')
                axs[idx,0].semilogy()
                # axs[idx,0].axvline(sampling_dt, color='k', lw=1.5, ls='-', alpha=.1)
                axs[idx,0].text(.98,.65, type(transformer).__name__, horizontalalignment='right', transform=axs[idx,0].transAxes)


    axs[0,0].set_title('processing time by step for predictions')
    axs[-1,0].set_xlabel('step time (ms)')

row_hist_per_step(run.pipeline, run, 0)


In [ ]:
run.pipeline.get_params()

In [ ]:
from adaptive_latents import Pipeline, CenteringTransformer, VanillaOnlineRegressor

x = np.arange(1)[:,None] + 1
y = x + 1


centerer = CenteringTransformer()
reg = VanillaOnlineRegressor()

for x_i, y_i in zip(x,y):
    data, stream = x_i, 0
    data, stream = centerer.partial_fit_transform(data, stream=stream, return_output_stream=True)
    data, stream = reg.partial_fit_transform(data, stream=stream, return_output_stream=True)

    data, stream = y_i, 1
    data, stream = centerer.partial_fit_transform(data, stream=stream, return_output_stream=True)
    data, stream = reg.partial_fit_transform(data, stream=stream, return_output_stream=True)

p = Pipeline([
    CenteringTransformer(),
    VanillaOnlineRegressor()
])

for x_i, y_i in zip(x,y):
    data, stream = x_i, 0
    data, stream = p.partial_fit_transform(data, stream=stream, return_output_stream=True)
    
    data, stream = y_i, 1
    data, stream = p.partial_fit_transform(data, stream=stream, return_output_stream=True)



In [ ]:
#TODO: reg.pft(_,_,True)

In [ ]:
print(p.steps[-1].c)
print(reg.c)

x, y

In [ ]:

from adaptive_latents import Pipeline, CenteringTransformer, VanillaOnlineRegressor

p = Pipeline([
    CenteringTransformer(),
    VanillaOnlineRegressor()
])

centerer = CenteringTransformer()
reg = VanillaOnlineRegressor()

x = np.array([[1]])
y = np.array([[2]])

output, stream = centerer.partial_fit_transform(x, stream=0, return_output_stream=True)
output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)

output, stream = centerer.partial_fit_transform(y, stream=1, return_output_stream=True)
output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)

p.partial_fit_transform(x, stream=0)
p.partial_fit_transform(y, stream=1);

# x = np.array([[1]])
# y = np.array([[2]])
# 
# output, stream = centerer.partial_fit_transform(x, stream=0, return_output_stream=True)
# output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)
# 
# output, stream = centerer.partial_fit_transform(y, stream=1, return_output_stream=True)
# output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)
# 
# p.partial_fit_transform(x, stream=0)
# p.partial_fit_transform(y, stream=1);
# 
# x = np.array([[3]])
# y = np.array([[4]])
# 
# output, stream = centerer.partial_fit_transform(x, stream=0, return_output_stream=True)
# output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)
# 
# output, stream = centerer.partial_fit_transform(y, stream=1, return_output_stream=True)
# output, stream = reg.partial_fit_transform(output, stream=stream, return_output_stream=True)
# 
# p.partial_fit_transform(x, stream=0)
# p.partial_fit_transform(y, stream=1);


In [ ]:
import sklearn.linear_model

regressor = sklearn.linear_model.LinearRegression()
regressor.fit(x,y)

import sklearn.pipeline
pipeline = sklearn.pipeline.Pipeline([('reg1', regressor)])
pipeline.predict(x)